In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("mnist_data/", one_hot=True)

Extracting mnist_data/train-images-idx3-ubyte.gz
Extracting mnist_data/train-labels-idx1-ubyte.gz
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Extracting mnist_data/t10k-labels-idx1-ubyte.gz


In [4]:
mnist.train.images.shape

(55000, 784)

In [5]:
X = tf.placeholder(tf.float32, shape=[None,784])
Y = tf.placeholder(tf.float32, shape=[None, 10])

In [6]:
X_tensor = tf.reshape(X, (-1, 28,28,1))

In [7]:
filter_size = 5
n_filters_in = 1
n_filters_out = 32

W_one = tf.get_variable(name='W',
                       shape=[filter_size, filter_size, n_filters_in, n_filters_out],
                       initializer=tf.random_normal_initializer())

b_one = tf.get_variable(name='B',
                       shape=[n_filters_out],
                       initializer=tf.constant_initializer())

In [8]:
h_one = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(X_tensor,
                  W_one,
                  strides=[1,2,2,1],
                  padding='SAME'), b_one))

In [9]:
n_filters_two_in = 32
n_filters_two_out = 64

W_two = tf.get_variable(name='W2',
                       shape=[filter_size, filter_size, n_filters_two_in, n_filters_two_out],
                       initializer=tf.random_normal_initializer())

b_two = tf.get_variable(name='b2',
                       shape=[n_filters_two_out],
                       initializer=tf.constant_initializer())

h_two = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(h_one,
                                              W_two,
                                              strides=[1,2,2,1],
                                              padding='SAME')
                                  ,b_two))

In [10]:
# flatten layer to plug into fully connected

h_two_flat = tf.reshape(h_two, [-1, 7*7*n_filters_two_out])

In [11]:
# fully connected layer

W_c = tf.get_variable(name='W_c',
                     shape=[h_two_flat.get_shape().as_list()[1], 128],
                      dtype=tf.float32,
                     initializer=tf.contrib.layers.xavier_initializer())

b_c = tf.get_variable(name='b_c',
                     shape=[128],
                      dtype=tf.float32,
                     initializer=tf.constant_initializer(0.0))

h_c = tf.nn.relu(tf.matmul(h_two_flat, W_c)+b_c)

In [12]:
W_f = tf.get_variable(name='W_f',
                     shape=[128, 10],
                      dtype=tf.float32,
                     initializer=tf.contrib.layers.xavier_initializer())

b_f = tf.get_variable(name='b_f',
                     shape=[10],
                      dtype=tf.float32,
                     initializer=tf.constant_initializer(0.0))

Y_pred = tf.nn.softmax(tf.matmul(h_c, W_f)+b_f)

In [13]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(Y_pred+ 1e-12)))
optimizer = tf.train.AdamOptimizer().minimize(cross_entropy)

In [14]:
correct_pred = tf.equal(tf.argmax(Y_pred, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, 'float'))

In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [16]:
batch_size = 100
n_epochs = 10

for i in range(n_epochs):
    for _ in range(500):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={X:batch_x, Y:batch_y})
    
    print("Cost:", sess.run(cross_entropy, feed_dict={X:mnist.train.images, Y:mnist.train.labels}))

print(sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))

Cost: 489992.0
Cost: 9155.97
Cost: 4156.91
Cost: 2136.89
Cost: 1698.0
Cost: 833.125
Cost: 1105.82
Cost: 1158.02
Cost: 1012.01
Cost: 944.796
0.9754
